In [ ]:
import pickle
import os
from flask import Flask, request, jsonify, render_template
from langchain.llms import OpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Flask app
app = Flask(__name__)

# Load the OpenAI API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')

# Load saved embeddings (assuming 'embeddings.pkl' contains 'documents' and 'embeddings')
with open('C:\Users\hrith\Desktop\content_last\document_embeddings_index.pkl', 'rb') as f:
    data = pickle.load(f)

documents = data['documents']
embeddings = data['embeddings']

# Initialize SentenceTransformer for embeddings (we assume embeddings are precomputed)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize FAISS vector store for efficient search
vectorstore = FAISS.from_documents(documents, embedding_model)

# Initialize Langchain with OpenAI LLM (using the API key)
llm = OpenAI(api_key=api_key, model="text-davinci-003")

# Initialize the Langchain agent for answering queries
agent = initialize_agent([llm], agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend_articles():
    """Recommend articles based on the content of the query."""
    query = request.json.get('query')

    # Perform similarity search using FAISS
    results = vectorstore.similarity_search(query, k=5)  # Top 5 most similar articles

    recommended_articles = [result['document'] for result in results]

    return jsonify({'recommendations': recommended_articles})

@app.route('/query', methods=['POST'])
def answer_query():
    """Answer user queries using the LLM (OpenAI model)."""
    query = request.json.get('query')

    # Use Langchain agent to answer the query
    response = agent.run(query)

    return jsonify({'answer': response})

if __name__ == '__main__':
    app.run(debug=True)